# Data Processing Notebook

This notebook is used to figure out data processing. Content will either be put into the final HW notebook
or a separate data processing .py file when it is finalized.

In [1]:
import numpy as np
import torch
from torch_geometric.data import Data
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from rdkit import Chem
from rdkit.Chem.rdmolops import GetAdjacencyMatrix
import pandas as pd

In [2]:
path = "hiv.csv"
data_pd = pd.read_csv(path)
data_pd

,smiles,HIV_active
0,CCC1=[O+][Cu-3]2([O+]=C(CC)C1)[O+]=C(CC)CC(CC)...,0
1,C(=Cc1ccccc1)C1=[O+][Cu-3]2([O+]=C(C=Cc3ccccc3...,0
2,CC(=O)N1c2ccccc2Sc2c1ccc1ccccc21,0
3,Nc1ccc(C=Cc2ccc(N)cc2S(=O)(=O)O)c(S(=O)(=O)O)c1,0
4,O=S(=O)(O)CCS(=O)(=O)O,0
...,...,...
41122,CCC1CCC2c3c([nH]c4ccc(C)cc34)C3C(=O)N(N(C)C)C(...,0
41123,Cc1ccc2[nH]c3c(c2c1)C1CCC(C(C)(C)C)CC1C1C(=O)N...,0
41124,Cc1ccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)C...,0
41125,Cc1cccc(N2C(=O)C3c4[nH]c5ccccc5c4C4CCC(C(C)(C)...,0


In [3]:
data_arr = np.array(data_pd)
data_arr.shape

(41127, 2)

# Processing SMILES String

This section deals with the processing

# Making torch Dataset
This section makes turns the dataset into a torch Dataset object for training

In [4]:
class HIVDataset(Dataset):
    def __init__(self, data_file, data_dir):
        self.data_dir = data_dir
        self.data_file = pd.read_csv(data_dir)
        # TODO: Finish __init__
    
    def __len__(self):
        raise NotImplemented
    
    def __getitem__(self, idx):
        raise NotImplemented
        